### import necessary toolsets

In [28]:
import json
import pandas as pd
import os
import string
import re # regular expression toolkit
import nltk # Natural language toolkit
from collections import defaultdict
from pandas.io.json import json_normalize
import requests

get data from steam website

In [29]:
api_key='3A629E52AE594CB777253431189F2967' # API key for getting information from Steam

In [30]:
def getownedgames(idnum):
    url='http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=3A629E52AE594CB777253431189F2967&steamid={}&format=json'.format(idnum)
    return url

### get data from folder and save file names

In [31]:
folder='E:/Desktop/jupyter/Capstone1-master/DATA/steam_reviews-master/steam_reviews-master/data/'

In [32]:
filenames=os.listdir(folder)

In [33]:
filenames[1]

'Arma_3.jsonlines'

### define function to get data from each file

In [34]:
def get_jsonline(file_name):
    data=[]
    with open(file_name) as f:
        for line in f:
            data.append(json.loads(line))
    return data

### make a first dataframe that will serve as frame for other data

In [35]:
data=json_normalize(get_jsonline(folder+filenames[1]))

In [36]:
review_df=pd.DataFrame(data)
print(review_df.columns)

Index(['achievement_progress.num_achievements_attained',
       'achievement_progress.num_achievements_percentage',
       'achievement_progress.num_achievements_possible', 'date_posted',
       'date_updated', 'found_helpful_percentage', 'friend_player_level',
       'num_badges', 'num_comments', 'num_found_funny', 'num_found_helpful',
       'num_found_unhelpful', 'num_friends', 'num_games_owned', 'num_groups',
       'num_guides', 'num_reviews', 'num_screenshots', 'num_voted_helpfulness',
       'num_workshop_items', 'orig_url', 'profile_url', 'rating', 'review',
       'review_url', 'steam_id_number', 'total_game_hours',
       'total_game_hours_last_two_weeks', 'username'],
      dtype='object')


In [37]:
# Only take columns that we will use
review_df=review_df[['rating','review','steam_id_number','total_game_hours','total_game_hours_last_two_weeks','username']]
print(review_df.tail())

           rating                                             review  \
7146  Recommended                                 Buy this game now.   
7147  Recommended  SIMPLY AMAZING. fun to play insurgency and kin...   
7148  Recommended  Arma 3 is very glitched ad horribly unoptimize...   
7149  Recommended  So once you've figured out what the  you're do...   
7150  Recommended  Reminds me a lot of DayZ. It doesn't have any ...   

        steam_id_number  total_game_hours  total_game_hours_last_two_weeks  \
7146       Chris_BKreme             392.5                              9.5   
7147  76561198088883739             270.9                              0.1   
7148         nitromen23             188.0                              8.8   
7149  76561198135659471             192.3                              0.0   
7150   darnude-sexstorm              13.6                              0.0   

                           username  
7146  xX69x420XxnNoScopesxX420x69Xx  
7147             Vladi

In [38]:
review_df['game_name']=filenames[1].split('.')[0]

### Fill up other game data to the dataframe

In [39]:
for name in filenames:
    if name.endswith(".jsonlines") and name != filenames[1]:
        data=json_normalize(get_jsonline(folder+name))
        df=pd.DataFrame(data)
        df=df[['rating','review','steam_id_number','total_game_hours','total_game_hours_last_two_weeks','username']]
        df['game_name']=name.split('.')[0]
        review_df=review_df.append(df, ignore_index=True, sort=False)
    else:
        continue

In [41]:
review_df.head(4)

,rating,review,steam_id_number,total_game_hours,total_game_hours_last_two_weeks,username,game_name
0,Recommended,My first game on A3 brought me the most horrif...,thisisthefallout,580.3,58.7,King Pootis,Arma_3
1,Recommended,This is not a game for people who want fast ac...,PeaceFaker,122.2,2.8,Peace,Arma_3
2,Recommended,Oh man. Where to even begin with this one. It ...,TheDanius,731.4,38.2,[13th] Danius,Arma_3
3,Recommended,This is quite possibly the most emotional shoo...,ArtificialApple,291.8,25.2,ArtificialApple,Arma_3


In [42]:
list(review_df)

['rating',
 'review',
 'steam_id_number',
 'total_game_hours',
 'total_game_hours_last_two_weeks',
 'username',
 'game_name']

## extract only the review section from each game data and clean them up for analysis

### extract words or bigrams from the reviews

In [43]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shshi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shshi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
raw_review=review_df['review']

In [45]:
raw_review.isnull().sum()

0

In [46]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = stopwords.words('english')+list(string.punctuation)+["game","games"]

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [49]:
for each_game in review_df['game_name'].unique():
    working_data=review_df.loc[(review_df['game_name'] == each_game)]

In [57]:
working_data.tail()

,rating,review,steam_id_number,total_game_hours,total_game_hours_last_two_weeks,username,game_name
7146,Recommended,Buy this game now.,Chris_BKreme,392.5,9.5,xX69x420XxnNoScopesxX420x69Xx,Arma_3
7147,Recommended,SIMPLY AMAZING. fun to play insurgency and kin...,76561198088883739,270.9,0.1,VladimirTheInhaler,Arma_3
7148,Recommended,Arma 3 is very glitched ad horribly unoptimize...,nitromen23,188.0,8.8,Nitromen23,Arma_3
7149,Recommended,So once you've figured out what the you're do...,76561198135659471,192.3,0.0,IntoTheGabe,Arma_3
7150,Recommended,Reminds me a lot of DayZ. It doesn't have any ...,darnude-sexstorm,13.6,0.0,IB6UB9,Arma_3


In [50]:
def vect(text):
    cvt=CountVectorizer(stop_words="english", analyzer='word')
    X=cvt.fit_transform(text)
    return X.toarray(), cvt.get_feature_names()

In [51]:
def tdfvect(text):   
    tfidf=TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word', stop_words="english")
    X=tfidf.fit_transform(text)
    return X.toarray(),tfidf.get_feature_names()

In [53]:
working_data=review_df.loc[(review_df['game_name'] == 'Arma_3')]
freq,words=vect(working_data['review'])
tdfscore,tdfwords=tdfvect(working_data['review'])

In [63]:
result=list(zip(tdfscore,tdfwords))

In [68]:
print(result[1][0].sum())

1094.8049968152


In [54]:
list(zip(words,freq.sum(axis=0)))

[('00', 16),
 ('000', 32),
 ('000000', 1),
 ('000000000001fps', 1),
 ('00035', 1),
 ('000ft', 1),
 ('008', 1),
 ('00usd', 1),
 ('0139011', 1),
 ('045', 1),
 ('05', 4),
 ('06', 4),
 ('061357', 1),
 ('07', 1),
 ('08', 2),
 ('09', 4),
 ('0ghz', 2),
 ('10', 2278),
 ('100', 160),
 ('1000', 54),
 ('10000', 3),
 ('100000', 3),
 ('1000000', 2),
 ('100000000000000', 1),
 ('1000e', 1),
 ('1000m', 1),
 ('1000man', 1),
 ('1000mhz', 1),
 ('1000s', 1),
 ('100fps', 1),
 ('100ft', 1),
 ('100g', 2),
 ('100gb', 1),
 ('100hrs', 1),
 ('100k', 1),
 ('100km', 2),
 ('100m', 9),
 ('100ms', 1),
 ('100x', 4),
 ('101', 2),
 ('101s', 1),
 ('1024', 1),
 ('1024x768', 1),
 ('103', 3),
 ('1037u', 1),
 ('104', 2),
 ('1050mhz', 1),
 ('106', 3),
 ('107410', 1),
 ('1080', 1),
 ('1080p', 9),
 ('10euros', 1),
 ('10fps', 7),
 ('10gb', 1),
 ('10k', 2),
 ('10km', 4),
 ('10lb', 1),
 ('10m', 5),
 ('10mil', 1),
 ('10million', 1),
 ('10mins', 1),
 ('10pm', 4),
 ('10s', 2),
 ('10x', 6),
 ('10ys', 1),
 ('11', 175),
 ('110', 7),
 ('

In [83]:
scoring=pd.DataFrame(tdfscore.sum(axis=0),index=tdfwords,columns=['score'])

In [84]:
scoring.head()

,score
00,115.775203
000,220.136807
000000,9.875007
000000000001fps,9.875007
00035,9.875007


In [87]:
scoring.sort_values(by='score',ascending=False)

,score
game,15118.547706
arma,10168.572761
like,6423.196366
10,6057.275997
play,5382.617826
just,4942.356105
good,4763.458456
great,4268.253403
fun,3881.335990
really,3737.511354


In [ ]:
print(sorted(scoring, key=lambda x: x[1], reverse=True))

In [19]:
cleaned_data=list()
commonword=list()
bigrams=list()

for num in range(len(raw_review)):
    review=raw_review.iloc[num]
    # split the words into 'tokens'
    cvt = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word')
    cv.fit(review)
    vector=cv.transform(review)
    cleaned_data.append(vector)

        
    # find the most frequent double word used in each review
    each_bigram=list()
    for wordnum in range(len(clean)-1):
        listitem=(clean[wordnum],clean[wordnum+1])
        each_bigram.append(listitem)
    bigrams.append(each_bigram)

NameError: name 'TfidfVectorizer' is not defined

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
working_data=review_df.loc[(review_df['game_name'] == 'Arma_3')]
#print(working_data['review'][1])
cv = CountVectorizer(stop_words="english", analyzer='word',ngram_range=(1,1), max_df=1.0, min_df=0.6, max_features=None)
cv.fit(working_data['review'])
vector=cv.transform(working_data['review'])

In [ ]:
print(cv.get_feature_names())

In [ ]:
working_data=review_df.loc[(review_df['game_name'] == 'Arma_3')]
#print(working_data['review'][1])
cvt = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word')
txt_fitted=cvt.fit(working_data['review'])
txt_transformed=txt_fitted.transform(working_data['review'])

In [ ]:
type(txt_fitted.get_feature_names())

In [ ]:
idf=cvt.idf_
scoring=list(zip(txt_fitted.get_feature_names(),idf))
print(sorted(scoring, key=lambda x: x[1], reverse=True))

In [ ]:
int(scoring[1][0])

In [ ]:
scoring_orig=scoring
print(scoring_orig[1][0])

In [ ]:
num_list=list()
num_list2=list()
for i in range(len(scoring)-1):
    try:
        int(scoring[i][0])
    except ValueError:
        num_list.append(scoring[i][0])
        num_list2.append(scoring[i][1])
        continue

In [ ]:
A_list=list(zip(num_list,num_list2))

In [ ]:
B_list=[item]

In [ ]:
raw_review[1]

In [ ]:
bigrams[len(cleaned_data)-1]

In [ ]:
raw_review[1]

In [ ]:
commonword[1]

In [ ]:
review_df['cleaned']=cleaned_data
review_df['frequent_words']=commonword
review_df['bigrams']=bigrams

In [ ]:
len(review_df)

In [ ]:
len(cleaned_data)

### Assign weight according to total playtime of each reviewer

In [88]:
review_weight=list()
for each_game in review_df['game_name'].unique():
    working_data=review_df.loc[(review_df['game_name'] == each_game)]
    review_weight.extend(working_data['total_game_hours']/max(working_data['total_game_hours']))
review_df['review_weight']=review_weight

In [89]:
review_df.head(3)

,rating,review,steam_id_number,total_game_hours,total_game_hours_last_two_weeks,username,game_name,review_weight
0,Recommended,My first game on A3 brought me the most horrif...,thisisthefallout,580.3,58.7,King Pootis,Arma_3,0.082158
1,Recommended,This is not a game for people who want fast ac...,PeaceFaker,122.2,2.8,Peace,Arma_3,0.017301
2,Recommended,Oh man. Where to even begin with this one. It ...,TheDanius,731.4,38.2,[13th] Danius,Arma_3,0.103551


### Pull out words according to the review weight

In [90]:
review_df['freq_word_values'] = review_df.apply(lambda row: list(row['frequent_words'].values()),axis=1)

KeyError: ('frequent_words', 'occurred at index 0')

In [ ]:
review_df['freq_word_keys'] = review_df.apply(lambda row: list(row['frequent_words'].keys()),axis=1)

In [ ]:
review_df['freq_word_weighted'] = review_df.apply(lambda row: [ i*row['review_weight']  for i in row['freq_word_values']],axis = 1)

In [ ]:
review_df['weighted_paired']=review_df.apply(lambda row: dict(zip(row['freq_word_keys'], row['freq_word_weighted'])),axis = 1)

In [ ]:
review_df.tail(3)

In [ ]:
review_df['weighted_paired'][1]

In [ ]:
key_df=pd.DataFrame(index=review_df['game_name'].unique(),columns=['key_word','key_bigram'])

In [ ]:
from collections import Counter
for each_game in review_df['game_name'].unique():
    working_data=review_df.loc[(review_df['game_name'] == each_game)]
    game_word=Counter()
    for pairs in working_data['weighted_paired']:
        game_word.update(pairs)
    key_df['key_word'][each_game]=game_word.most_common()

In [ ]:
bigram=defaultdict()
for each_game in review_df['game_name'].unique():
    working_data=review_df.loc[(review_df['game_name'] == each_game)]
    wordfreq[word]+=1

In [ ]:
key_df

In [ ]:
for each_game in review_df['game_name'].unique():
    working_data=review_df.loc[(review_df['game_name'] == each_game)]
    

In [ ]:
key_df['key_word']['Team_Fortress_2'][1:30]

In [ ]:
key_df['key_word']['Sid_Meiers_Civilization_5']

In [ ]:
len(working_data)

In [ ]:
sorted_by_value = sorted(game_word.items(), key=lambda kv: kv[1],reverse=True)

In [ ]:
print(sorted_by_value)

In [ ]:
from pattern.en import sentiment

In [ ]:
sentiment(raw_review[79436]).assessments[1]

In [ ]:
sentiment(raw_review[79436])

In [ ]:
key_df

In [ ]:
from collections import OrderedDict
result=defaultdict(int)
for each_game in review_df['game_name'].unique():
    working_data=review_df.loc[(review_df['game_name'] == 'Warframe')]
    for num in range(len(working_data['ranked_words'])):
        for word in working_data['ranked_words'].iloc[num]:
            result[word[1]]+=word[0]
    frequent_words=OrderedDict(sorted(result.items(), key=lambda t:t[1], reverse=True))
    for (key, value) in enumerate(frequent_words.items()):
        key_df['key_word']['Warframe']=frequent_words[0:100]
    #x=((value*working_data.review_weight,key) for key,value in commonword.items())   
    #ranked.extend(x)

In [ ]:
print((key for key, value in frequent_words.items()))

In [ ]:
type(key_df['key_word']['Warframe'])

In [ ]:
key_df

In [ ]:
result = defaultdict(int)

for d in lst:
    result[d['name']] += int(d['value'])

In [ ]:
print(sorted(ranked,reverse=True)[1:100])

In [ ]:
review_df.head(3)

In [ ]:
df.head()

In [ ]:
print(weighted_list)

In [ ]:
review1=pd.DataFrame(reviewdata['Warframe'])

In [ ]:
s=sum(review1['total_game_hours'])

In [ ]:
type([review1['total_game_hours']/s])

In [ ]:
reviewex=reviewdata['Warframe'][99]['review']
print(reviewex)

In [ ]:
cleaned=reviewdata['Warframe'][99]['cleaned review']
print(cleaned)

In [ ]:
print(reviewdata['Warframe'][99]['frequent words'])

In [ ]:
wordfreq=defaultdict(int)

In [ ]:
wordfreq.items()

In [ ]:
type(wordfreq)

In [47]:
cleaned_data=list()
commonword=list()
bigrams=list()

for num in range(len(raw_review)):
    review=raw_review.iloc[num]
    # split the words into 'tokens'
    tokens=word_tokenize(review)
    # lowercase every tokens
    lower_tokens=[token.lower() for token in tokens]
    # remove stopwords and punctuation from the words
    words=[word for word in lower_tokens if word not in stop_words]
    clean=[str(word) for word in words if word.isalpha()]
    cleaned_data.append(clean)
    
    # find the most frequent single word used in each review
    wordfreq=defaultdict(int)
    for word in clean:
        wordfreq[word]+=1
    commonword.append(wordfreq)
        
    # find the most frequent double word used in each review
    each_bigram=list()
    for wordnum in range(len(clean)-1):
        listitem=(clean[wordnum],clean[wordnum+1])
        each_bigram.append(listitem)
    bigrams.append(each_bigram)